### SOLUTION:
Create an unsupervised scientific literature understanding system that can take in common terms and analyze a very large corpus of scientific papers and return highly relevant text fragments from papers containing topical data relating to the common text inputed, allowing a single researcher or small team to gather targeted information and quickly and easily locate relevant text in the scientific papers to answer important questions about the new virus from a large corpus of documents.

### APPROACH: The newest implementation outlined:
- Open the metadata CSV file in a dataframe
- Drop the duplicates by abstract
- Loop (comprehension) a list of lists for the keyword for searches. eg. search=[['incubation','period','range']]
- Stem the keywords so incubation becomes incubat etc.
- Append indpendently - different variations of corona references - (covid, cov, etc) to the keywords avoids pulling older research
- Query function in pandas to query the above terms with the keywords - the abstract has to have all the keywords and at least one of (covid, cov, etc)
- Drop the duplicates in the df again after the queries
- Caclulate a relevance score for all the abstracts in the query result df
- raw_score = total count of the keywords in the abstract
- final_score = (raw_score/len(abstract))*raw_score
- Sort the df on the final_score ascending=False (best on top)
- Drop rows with a relevance score < .02
- Parse the relevant abstarcts into sentences split('. ') works well.
- Test if keywords are in the sentences
- If sentence has all keywords, it is added to the df for display.
- If you are seeking statistics in the data, adding % to the search terms works well
- df with search results displayed in HTML - currently limiting results to 3 for ease of scanning

In [272]:
# fit the notebook to browser
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [273]:
# importing all necessary libraries
import numpy as np
import pandas as pd
import functools
from nltk import PorterStemmer

In [274]:
# avoiding error while printing output
pd.options.mode.chained_assignment = None

# reading metadata.csv
df=pd.read_csv("/home/varun_nagrare/2020-08-03/metadata.csv", low_memory=False, usecols=['title','journal','abstract','authors','doi','publish_time'])
df.head()

,title,doi,abstract,publish_time,authors,journal
0,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,OBJECTIVE: This retrospective chart review des...,2001-07-04,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis
1,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,Inflammatory diseases of the respiratory tract...,2000-08-15,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res
2,Surfactant protein-D and pulmonary host defense,10.1186/rr19,Surfactant protein-D (SP-D) participates in th...,2000-08-25,"Crouch, Erika C",Respir Res
3,Role of endothelin-1 in lung disease,10.1186/rr44,Endothelin-1 (ET-1) is a 21 amino acid peptide...,2001-02-22,"Fagan, Karen A; McMurtry, Ivan F; Rodman, David M",Respir Res
4,Gene expression in epithelial cells in respons...,10.1186/rr61,Respiratory syncytial virus (RSV) and pneumoni...,2001-05-11,"Domachowske, Joseph B; Bonville, Cynthia A; Ro...",Respir Res


In [275]:
# keep only documents with covid alternate names
def search_focus(df):
    dfa = df[df['abstract'].str.contains('covid')]
    dfb = df[df['abstract'].str.contains('-cov-2')]
    dfc = df[df['abstract'].str.contains('cov2')]
    dfd = df[df['abstract'].str.contains('ncov')]
    dfe = df[df['abstract'].str.contains('coronavirus')]
    dff = df[df['abstract'].str.contains('sars-cov-2')]
    
    frames=[dfa,dfb,dfc,dfd,dfe,dff]
    df = pd.concat(frames)
    df=df.drop_duplicates(subset='title', keep="first")
    return df

In [276]:
print (df.shape)

#drop duplicates
df=df.drop_duplicates()

#drop NANs 
df=df.fillna("no data ")

# drop duplicates keeping the first occurence
df=df.drop_duplicates(subset='title', keep="first")

# finding publish_time which contains year 2019 and 2020
df=df[df['publish_time'].str.contains('2019', case=False, regex=False) | df['publish_time'].str.contains('2020', case=False, regex=False)]

(207311, 6)


In [277]:
# checking if data is fetched as intended
df.head()

,title,doi,abstract,publish_time,authors,journal
3502,Human rhinovirus spatial-temporal epidemiology...,10.12688/wellcomeopenres.14836.2,Background: Human rhinovirus (HRV) is the pred...,2019-03-27,"Morobe, John Mwita; Nyiro, Joyce U.; Brand, Sa...",Wellcome Open Res
3587,Multiple Introductions of Domestic Cat Feline ...,10.3201/eid2501.181347,The endangered Florida panther (Puma concolor ...,2019-01-03,"Chiu, Elliott S.; Kraberger, Simona; Cunningha...",Emerg Infect Dis
3606,Impact of glucose-6-phosphate dehydrogenase de...,10.1371/journal.pone.0209204,Glucose-6-phosphate dehydrogenase (G6PD) defic...,2019-01-02,"May, Win Lai; Kyaw, Myat Phone; Blacksell, Stu...",PLoS One
3618,Improvement in the survival rates of extracorp...,10.1186/s13054-018-2293-5,BACKGROUND: Although the utilization of extrac...,2019-01-03,"Baek, Moon Seong; Lee, Sang-Min; Chung, Chi Ry...",Crit Care
3619,Mental health workers perceptions of disaster ...,10.1186/s12889-018-6313-9,BACKGROUND: The post-disaster mental health cr...,2019-01-03,"Xi, Yingjun; Chen, Runsen; Gillespie, Amy L.; ...",BMC Public Health


In [278]:
# checking if data is fetched as intended
df.tail()

,title,doi,abstract,publish_time,authors,journal
207295,Impact of COVID‐19 on the mental health of sur...,10.1002/hed.26291,Unprecedented times call for extraordinary mea...,2020-05-28,"Balasubramanian, Anusha; Paleri, Vigneshwar; B...",Head Neck
207301,Assessment of COVID-19 Waste Flows During the ...,10.3390/ijerph17155439,This paper provides a rapid assessment method ...,2020,"Mihai, Florin-Constantin",Int. J. Environ. Res. Public Health
207303,Extended use of face masks during the COVID-19...,10.1016/j.polymdegradstab.2020.109251,The current COVID-19 pandemic has resulted in ...,2020-06-04,"Celina, Mathew; Martinez, Estevan; Omana, Mich...",Polym Degrad Stab
207304,Containment of future waves of COVID-19: simul...,10.1101/2020.06.05.20123372,We used multi-agent simulations to estimate th...,2020-06-07,"Fiore, Vincenzo G.; DeFelice, Nicholas; Glicks...",medRxiv
207309,"Examining Social Determinants of Health, Stigm...",10.3390/healthcare8020168,There is growing attention to disparities in t...,2020-06-12,"Turner-Musa, Jocelyn; Ajayi, Oluwatoyin; Kemp,...",Healthcare (Basel)


In [279]:
# convert abstracts to lowercase
df["abstract"] = df["abstract"].str.lower()+df["title"].str.lower()

#show 5 lines of the new dataframe
df=search_focus(df)

print(df.shape)

df.head()

(72535, 6)


,title,doi,abstract,publish_time,authors,journal
4662,Latest assessment on COVID-19 from the Europea...,10.2807/1560-7917.es.2020.25.8.2002271,no data latest assessment on covid-19 from the...,2020-02-27,no data,Euro Surveill
4698,Updated rapid risk assessment from ECDC on the...,10.2807/1560-7917.es.2020.25.9.2003051,no data updated rapid risk assessment from ecd...,2020-03-05,no data,Euro Surveill
4732,Updated rapid risk assessment from ECDC on the...,10.2807/1560-7917.es.2020.25.10.2003121,no data updated rapid risk assessment from ecd...,2020-03-12,no data,Euro Surveill
4800,Empfehlungen zur intensivmedizinischen Therapi...,10.1007/s00063-020-00674-3,no data empfehlungen zur intensivmedizinischen...,2020-03-12,"Kluge, Stefan; Janssens, Uwe; Welte, Tobias; W...",Med Klin Intensivmed Notfmed
5683,The impact of COVID-19 on the provision of don...,10.1038/s41409-020-0873-x,no data the impact of covid-19 on the provisio...,2020-03-23,"Szer, Jeff; Weisdorf, Daniel; Querol, Sergio; ...",Bone Marrow Transplant


In [280]:
# function to find the root word of search_words
def stem_words(search_words):
    stemmer = PorterStemmer()
    singles=[]
    for w in search_words:
        singles.append(stemmer.stem(w))
    return singles

In [281]:
# function to find exact words
def search_dataframe(df,search_words):
    search_words=stem_words(search_words)
    df1=df[functools.reduce(lambda a, b: a&b, (df['abstract'].str.contains(s) for s in search_words))]
    return df1

In [282]:
# function analyze search results for relevance with word count or abstract length
def search_relevance(rel_df,search_words):
    rel_df['score']=""
    search_words=stem_words(search_words)
    for index, row in rel_df.iterrows():
        abstract = row['abstract']
        result = abstract.split()
        len_abstract=len(result)
        score=0
        for word in search_words:
            score=score+result.count(word)
        final_score=(score/len_abstract)
        rel_score=score*final_score
        rel_df.loc[index, 'score'] = rel_score
    rel_df=rel_df.sort_values(by=['score'], ascending=False)
    return rel_df

In [283]:
# function to get best sentences from the search results
def get_sentences(df1,search_words):
    # create a new dataframe
    df_table = pd.DataFrame(columns = ["pub_date","authors","title","fragment","rel_score"])
    search_words=stem_words(search_words)
    
    for index, row in df1.iterrows():
        pub_sentence=''
        sentences_used=0
        #break apart the absracrt to sentence level
        sentences = row['abstract'].split('. ')
        
        #loop through the sentences of the abstract
        for sentence in sentences:
            # missing lets the system know if all the words are in the sentence
            missing=0
            #loop through the words of sentence
            for word in search_words:
                #if keyword missing change missing variable
                if word not in sentence:
                    missing=1
                
                #if '%' in sentence:
                    #missing=missing-1
            # after all sentences processed show the sentences not missing keywords
            if missing==0 and len(sentence)<1000 and sentence!='':
                sentence=sentence.capitalize()
                if sentence[len(sentence)-1]!='.':
                    sentence=sentence+'.'
                pub_sentence=pub_sentence+'<br><br>'+sentence
        
        if pub_sentence!='':
            sentence=pub_sentence
            sentences_used=sentences_used+1
            authors=row["authors"].split(" ")
            link=row['doi']
            title=row["title"]
            score=row["score"]
            linka='https://doi.org/'+link
            linkb=title
            sentence='<p fontsize=tiny" align="left">'+sentence+'</p>'
            final_link='<p align="left"><a href="{}">{}</a></p>'.format(linka,linkb)
            to_append = [row['publish_time'],authors[0]+' and others.',final_link,sentence,score]
            df_length = len(df_table)
            df_table.loc[df_length] = to_append
    return df_table

# What is known about transmission, incubation, and environmental stability?

In [284]:
# list of lists of search terms
questions=[
['Q: What is the range of incubation periods for the disease in humans?'],
['Q: How long are individuals contagious?'],
['Q: How long are individuals contagious, even after recovery.'],
['Q: Does the range of incubation period vary across age groups?'],
['Q: Does the range of incubation period vary with children?'],
['Q: Does the range of incubation period vary based on underlying health?'],
['Q: What do we know about the basic reproduction number?'],
['Q: What is the prevalance of asymptomatic transmission?'],
['Q: What do we know about asymptomatic transmission in children?'],
['Q: What do we know about seasonality of transmission?'],
['Q: Informing decontamination based on physical science of the coronavirus?'],
['Q: What do we know about stability of the virus in environmental conditions?'],
['Q: What do we know about persistence of the virus on various substrates? (e.g., nasal discharge, sputum, urine, fecal matter, blood)'],
['Q: What do we know about persistence of the virus on various surfaces? (e,g., copper, stainless steel, plastic) '],
['Q: What do we know about viral shedding duration?'],
['Q: What do we know about viral shedding in fecal/stool?'],
['Q: What do we know about viral shedding from nasopharynx?'],
['Q: What do we know about viral shedding in blood?'],
['Q: What do we know about viral shedding in urine?'],
['Q: What do we know about implemtation of diagnostics?'],
['Q: What do we know about disease models?'],
['Q: What do we know about models for disease infection?'],
['Q: What do we know about models for disease transmission?'],
['Q: Are there studies about phenotypic change?'],
['Q: What is know about adaptations (mutations) of the virus?'],
['Q: What do we know about the human immune response and immunity?'],
['Q: Is population movement control effective in stopping transmission?'],
['Q: Effectiveness of personal protective equipment (PPE)?'],
['Q: What is the role of environment in transmission?']
]

search_words=[
['incubation','period','range'],
['viral','shedding','duration'],
['asymptomatic','shedding'],
['incubation','period','age','statistically','significant'],
['incubation','period','child'],
['incubation','groups','risk'],
['basic','reproduction', 'number','%'],
['asymptomatic','infection','%'],
['asymptomatic','children'],
['seasonal','transmission'],
['contaminat','object'],
['environmental', 'conditions'],
['sputum','stool','blood','urine'],
['persistence','surfaces'],
['duration','viral','shedding'],
['shedding','stool'],
['shedding','nasopharynx'],
['shedding','blood'],
['shedding','urine'],
['diagnostics','point'],
['model', 'disease'],        
['model', 'infection'],
['model', 'transmission'],
['phenotypic'],
['mutation'],
['immune','response'],
['restriction', 'movement'],
['protective','clothing'],
['transmission','routes']
]

q_count=0

for search_words in search_words:
    str1=''
    # make a string of the search words to print readable table
    str1=''.join(questions[q_count])
    
    #search the dataframe for all words
    df1=search_dataframe(df,search_words)

    # analyze search results for relevance 
    df1=search_relevance(df1,search_words)

    # get best sentences
    df_table=get_sentences(df1,search_words)
    
    # get the length of rows
    length=df_table.shape[0]
    
    #limit 3 results
    df_table=df_table.head()
    
    # dropping the rel_score
    df_table=df_table.drop(['rel_score'], axis=1)
    
    #convert df to html readable table
    df_table=HTML(df_table.to_html(escape=False,index=False))
    
    # display the question
    display(HTML('<br><h3>'+str1+'</h3>'))
    
    #display the summary
    if length<1:
        print ("No reliable answer found")
    else:
        display(df_table)
    q_count=q_count+1

pub_date,authors,title,fragment
2020-03-18,"Jiang, and others.",Is a 14-day quarantine period optimal for effectively controlling coronavirus disease 2019 (COVID-19)?,Results the full range of incubation periods of the covid-19 cases ranged from 0 to 33 days among 2015 cases.
2020-05-15,"Bui, and others.",Estimation of the incubation period of SARS-CoV-2 in Vietnam,"Average incubation periods estimated using different distribution model ranged from 6.0 days to 6.4 days with the weibull distribution demonstrated the best fit to the data.The estimated mean of incubation period using weibull distribution model was 6.4 days (95% credible interval (ci): 4.89 - 8.5), standard deviation (sd) was 3.05 (95%ci 3.05 - 5.30), median was 5.6, ranges from 1.35 to 13.04 days (2.5th to 97.5th percentiles)."
2020-03-08,"Xia, and others.",Transmission of corona virus disease 2019 during the incubation period may lead to a quarantine loophole,"Results: the estimated mean incubation period for covid-19 was 4.9 days (95% confidence interval [ci], 4.4 to 5.4) days, ranging from 0.8 to 11.1 days (2.5th to 97.5th percentile)."
2020,"Yang, and others.","Estimation of incubation period and serial interval of COVID-19: analysis of 178 cases and 131 transmission chains in Hubei province, China","Our estimated median incubation period of covid-19 is 5.4 days (bootstrapped 95% confidence interval (ci) 4.8-6.0), and the 2.5th and 97.5th percentiles are 1 and 15 days, respectively; while the estimated serial interval of covid-19 falls within the range of -4 to 13 days with 95% confidence and has a median of 4.6 days (95% ci 3.7-5.5)."
2020,"Yang, and others.",[The preliminary analysis on the characteristics of the cluster for the COVID-19],"We selected 325 cases to estimate the incubation period and its range was 1 to 20 days, median was 7 days, and mode was 4 days."


pub_date,authors,title,fragment
2020-05-23,"Weiss, and others.",Spatial and temporal dynamics of SARS-CoV-2 in COVID-19 patients: A systematic review,"In this study, we aimed to provide a coherent overview from published studies of the duration of viral detection and viral load in covid-19 patients, stratified by specimen type, clinical severity and age."
2020,"Dodds, and others.",Model-Informed Drug Repurposing: Viral Kinetic Modeling to Prioritize Rational Drug Combinations for COVID-19,"The endpoints and metrics included viral load area under the curve (auc), duration of viral shedding, and epithelial cells infected.In addition, we observed that the time-window opportunity for a therapeutic intervention to effect duration of viral shedding exceeds the effect on sparing epithelial cells from infection or impact on viral load auc."
2020-07-21,"Dodds, and others.",Model-Informed Drug Repurposing: Viral Kinetic Modeling to Prioritize Rational Drug Combinations for COVID-19.,"The endpoints and metrics included viral load area under the curve (auc), duration of viral shedding, and epithelial cells infected.In addition, we observed that the time-window opportunity for a therapeutic intervention to effect duration of viral shedding exceeds the effect on sparing epithelial cells from infection or impact on viral load auc."
2020-07-30,"Park, and others.",Determining the period of communicability of SARS-CoV-2: A rapid review of the literature,"Methods: studies reporting empirical data on the period of communicability of sars-cov-2 through investigations of duration of communicability based on in-person contact ('contact transmission'), isolation and culture of virus ('viral isolation'), and viral shedding by detection of nucleic acids by rt-pcr ('viral shedding') were identified through searches of peer-reviewed and pre-print health sciences literature databases (ovid medline, embase, google scholar, medrxiv and arxiv) and the grey literature."
2020-07-28,"Cevik, and others.","SARS-CoV-2 viral load dynamics, duration of viral shedding and infectiousness: a living systematic review and meta-analysis","Background viral load kinetics and the duration of viral shedding are important determinants for disease transmission.We aim i) to characterise viral load dynamics, duration of viral rna, and viable virus shedding of sars-cov-2 in various body fluids and ii) to compare sars-cov-2 viral dynamics with sars-cov-1 and mers-cov.Methods: medline, embase, europe pmc, preprint servers and grey literature were searched to retrieve all articles reporting viral dynamics and duration of sars-cov-2, sars-cov-1 and mers-cov shedding.Funding: no funding was received.sars-cov-2 viral load dynamics, duration of viral shedding and infectiousness: a living systematic review and meta-analysis."


pub_date,authors,title,fragment
2020-03-31,"Abduljalil, and others.","Epidemiology, genome, and clinical features of the pandemic SARS-CoV-2: a recent view",Asymptomatic carrier state is of paramount importance because of carriers' ability to spread the infection and to shed the virus into the air and surroundings.
2020,"Li, and others.",Viral shedding dynamics in asymptomatic and mildly symptomatic patients infected with SARS-CoV-2,"The median duration of viral shedding was 11.5, 28 and 31 days for pre-symptomatic, asymptomatic and mild symptomatic patients, separately.Conclusions: long-term viral shedding was presented in patients with mild symptoms and asymptomatic cases, and specific antibody production to sars-cov-2 may not guarantee viral clearance after discharge.These observations should be considered when making decisions regarding clinical and public health and strategies for prevention and control of sars-cov-2 infection.viral shedding dynamics in asymptomatic and mildly symptomatic patients infected with sars-cov-2."
2020,"Matava, and others.",Pediatric Airway Management in COVID-19 Patients: Consensus Guidelines From the Society for Pediatric Anesthesia's Pediatric Difficult Intubation Collaborative and the Canadian Pediatric Anesthesia Society,"Pedi-c identified overarching goals during care, including minimizing aerosolized respiratory secretions, minimizing the number of clinicians in contact with a patient, and recognizing that undiagnosed asymptomatic patients may shed the virus and infect health care workers."
2020-04-20,"Matava, and others.",Pediatric Airway Management in Coronavirus Disease 2019 Patients: Consensus Guidelines From the Society for Pediatric Anesthesia’s Pediatric Difficult Intubation Collaborative and the Canadian Pediatric Anesthesia Society,"Pedi-c identified overarching goals during care, including minimizing aerosolized respiratory secretions, minimizing the number of clinicians in contact with a patient, and recognizing that undiagnosed asymptomatic patients may shed the virus and infect health care workers."
2020-05-25,"Beggs, and others.",Is there an airborne component to the transmission of COVID-19? : a quantitative analysis study,"Conclusions given that live sars-cov-2 virions are known to be shed in high concentrations from the nasal cavity of both symptomatic and asymptomatic covid-19 patients, the results suggest that individuals who share enclosed spaces with an infector may be at risk of contracting covid-19 by the aerosol route, even when practicing social distancing.is there an airborne component to the transmission of covid-19? : a quantitative analysis study."


pub_date,authors,title,fragment
2020-02-29,"Han, and others.",Estimate the incubation period of coronavirus 2019 (COVID-19),We found that the incubation periods of the groups with age>=40 years and age<40 years demonstrated a statistically significant difference.


pub_date,authors,title,fragment
2020-03-18,"Jiang, and others.",Is a 14-day quarantine period optimal for effectively controlling coronavirus disease 2019 (COVID-19)?,The median incubation period of both male and female adults was similar (7-day) but significantly shorter than that (9-day) of child cases (p=0.02).
2020,"Merino-Navarro, and others.",Prevención y tratamiento del Covid-19 en la población pediátrica desde una perspectiva familiar y comunitaria./ [Prevention and treatment of Covid-19 in the pediatric population from the family and community perspective],Our objective is to analyze the scientific evidence on the specific recommendations for pediatric care in cases of covid-19 from the family and community settings.the main recommendations and preventive measures in primary health care settings and at home have been selected and analyzed from an integrative approach that includes the biopsychosocial aspects of the child during confinement.the importance of caring for children in the face of the disease lies above all in ensuring the correct measures for the prevention of contagion due to the condition of acting as possible carriers during an incubation period of up to 21 days.
2020-05-16,"Merino-Navarro, and others.",Prevención y tratamiento del Covid-19 en la población pediátrica desde una perspectiva familiar y comunitaria,The importance of caring for children in the face of the disease lies above all in ensuring the correct measures for the prevention of contagion due to the condition of acting as possible carriers during an incubation period of up to 21 days.
2020,"Liu, and others.",Risk factors associated with COVID-19 infection: a retrospective cohort study based on contacts tracing,"Children, old people, females, and family members are susceptible of covid-19 infection, while index cases in the incubation period had lower contagiousness."
2020-07-01,"Liu, and others.",Risk factors associated with COVID-19 infection: a retrospective cohort study based on contacts tracing.,"Conclusion children, old people, females and family members are susceptible to be infected with covid-19, while index cases in incubation period had lower contagiousness."


...

pub_date,authors,title,fragment
2020-04-30,"Padhi, and others.",Laboratory Diagnosis of Novel Coronavirus Disease 2019 (COVID-19) Infection,"Furthermore, nucleic acid sequencing may be done for the identification of mutation in the genome of sars-cov-2."
2020-07-06,"Graudenzi, and others.",Mutational Signatures and Heterogeneous Host Response Revealed Via Large-Scale Characterization of SARS-COV-2 Genomic Diversity,"To dissect the mechanisms underlying the observed inflation of variants in sars-cov-2 genome, we present the largest up-to-date analysis of intra-host genomic diversity, which reveals that the majority of samples present a complex sublineage architecture, due to the interplay between host-related mutational processes and transmission dynamics.Strikingly, the deconvolution of the entire set of intra-host variants reveals the existence of mutually exclusive viral mutational signatures, which prove that distinct hosts differently respond to sars-cov-2 infections.In particular, two signatures are likely ruled by apobec and reactive oxygen species (ros), which induce hypermutation in a significant number of samples, and appear to be affected by severe purifying selection.Conversely, several mutations linked to low-rate mutational processes appear to transit to clonality in the population, eventually leading to the definition of new viral genotypes and to an increase of overall genomic diversity.Finally, we demonstrate that a high number of variants are observed in samples associated to independent lineages, likely due to signature-related mutational hotspots or to positive selection.mutational signatures and heterogeneous host response revealed via large-scale characterization of sars-cov-2 genomic diversity."
2020-04-29,"Rose, and others.",Intra-host site-specific polymorphisms of SARS-CoV-2 is consistent across multiple samples and methodologies,"While mutations resulting from amplification and/or sequencing errors cannot be excluded, the observation of shared polymorphic sites with high maf across multiple samples and sequencing methods is consistent with true underlying variation."
2020-05-26,"Phelan, and others.","Controlling the SARS-CoV-2 outbreak, insights from large scale whole genome sequences generated across the world","We also noted that convergent evolution has potentially occurred across several locations in the genome, showing selection pressures, including on the spike glycoprotein where we noted a potentially critical mutation that could affect its binding to the ace2 receptor.We also report on mutations that could prevent current molecular diagnostics from detecting some of the sub-clades."
2020-03-12,"Su, and others.",Discovery of a 382-nt deletion during the early evolution of SARS-CoV-2,"Overwhelmingly these viruses had mutations or deletions in orf8, that have been associated with reduced replicative fitness of the virus [2]."


pub_date,authors,title,fragment
2020-07-23,"Taqarort, and others.",Vitamine D et risque des infections respiratoires aiguës : grippe et COVID-19,Vitamin d appears to stimulate the activation of the immune response and limit the destructive consequences of immunopathology induced by pathogens.
2020-05-21,"Schijns, and others.",Prevention and treatment of COVID‐19 disease by controlled modulation of innate immunity,"Since it will be some time until a safe and effective vaccine will be available, the immediate priority is to harness innate immunity to accelerate early antiviral immune responses."
2020-06-08,"Marcault, and others.",In response to the letter of Montagud‐Marrahi et al.,"Moreover, as innate immunity is altered by the infection, decreased doses of immunosuppressive drugs should be considered.in response to the letter of montagud‐marrahi et al."
2020-07-14,"Gebicki, and others.",COVID-19 infection: mitohormetic concept of immune response,No data covid-19 infection: mitohormetic concept of immune response.
2020-06-10,"Suwanwongse, and others.",Hyperpyrexia in COVID‐19 patients,"We propose it may be caused by sars‐cov‐2 related brain injury, exuberant immune response, and thrombus formation."


pub_date,authors,title,fragment
2020,"Guerrero, and others.",Canadian children's and youth's adherence to the 24-h movement guidelines during the COVID-19 pandemic: A decision tree analysis,"Results highlight the importance of targeting parents' perceived capability for the promotion of children's and youth's movement behaviors during challenging times of the covid-19 pandemic, paying particular attention to enhancing parental perceived capability to restrict screen time.canadian children's and youth's adherence to the 24-h movement guidelines during the covid-19 pandemic: a decision tree analysis."
2020,"Moore, and others.",Impact of the COVID-19 virus outbreak on movement and play behaviours of Canadian children and youth: a national survey,"Though the covid-19 virus outbreak has changed the daily lives of children and youth, it is unknown to what extent related restrictions may compromise the ability to play and meet movement behaviour recommendations.This secondary data analysis examined the immediate impacts of covid-19 restrictions on movement and play behaviours in children and youth.Results: only 4.8% (2.8% girls, 6.5% boys) of children and 0.6% (0.8% girls, 0.5% boys) of youth were meeting combined movement behaviour guidelines during covid-19 restrictions."
2020-06-13,"Tang, and others.",Movement control as an effective measure against Covid-19 spread in Malaysia: an overview,"Results: at the onset of the covid-19 outbreak, malaysia had initiated travel restrictions and quarantine; but with a persistent increase in new covid-19 cases, the movement control order was finally rolled out on 18 march 2020, requiring closure of all businesses except those providing essential services and items."
2020-06-12,"Bucsky, and others.",Modal share changes due to COVID-19: The case of Budapest,"One of the most important responses of countries worldwide to slow the spread of the pandemic has been to restrict the movement of people, which has had a considerable effect on transport systems."
2020-07-21,"Ong, and others.",Implications of the COVID-19 lockdown on dengue transmission and the occurrence of Aedes aegypti (Linnaeus) and Aedes albopictus (Skuse) in Malaysia,"The impact of movement restrictions (mrs) during the covid-19 lockdown on the existing endemic infectious disease dengue fever has generated considerable research interest.With a partial lockdown implemented by malaysia on 18 march, we postulate the movement restrictions (mrs) of people in large-scale would hamper the regular dengue transmission and aim to reveal the impact of mrs on both dengue incidences and aedes mosquitoes."


pub_date,authors,title,fragment
2020,"Chughtai, and others.",Effectiveness of Cloth Masks for Protection Against Severe Acute Respiratory Syndrome Coronavirus 2,"Cloth masks have been used in healthcare and community settings to protect the wearer from respiratory infections.The filtration effectiveness of cloth masks is generally lower than that of medical masks and respirators; however, cloth masks may provide some protection if well designed and used correctly.Multilayer cloth masks, designed to fit around the face and made of water-resistant fabric with a high number of threads and finer weave, may provide reasonable protection.In community settings, however, cloth masks may be used to prevent community spread of infections by sick or asymptomatically infected persons, and the public should be educated about their correct use.effectiveness of cloth masks for protection against severe acute respiratory syndrome coronavirus 2."
2020-07-08,"Chughtai, and others.",Effectiveness of Cloth Masks for Protection Against Severe Acute Respiratory Syndrome Coronavirus 2.,"Cloth masks have been used in healthcare and community settings to protect the wearer from respiratory infections.The filtration effectiveness of cloth masks is generally lower than that of medical masks and respirators; however, cloth masks may provide some protection if well designed and used correctly.Multilayer cloth masks, designed to fit around the face and made of water-resistant fabric with a high number of threads and finer weave, may provide reasonable protection.In community settings, however, cloth masks may be used to prevent community spread of infections by sick or asymptomatically infected persons, and the public should be educated about their correct use.effectiveness of cloth masks for protection against severe acute respiratory syndrome coronavirus 2."
2020-07-17,"Fisher, and others.","Factors Associated with Cloth Face Covering Use Among Adults During the COVID-19 Pandemic - United States, April and May 2020.","The use of a cloth face covering was associated with theory-derived constructs that indicate a favorable attitude toward them, intention to use them, ability to use them, social support for using them, and beliefs that they offered protection for self, others, and the community."
2020,"Zangmeister, and others.",Filtration Efficiencies of Nanoscale Aerosol by Cloth Mask Materials Used to Slow the Spread of SARS-CoV-2,"Filtration efficiency (fe), differential pressure (Δp), quality factor (qf), and construction parameters were measured for 32 cloth materials (14 cotton, 1 wool, 9 synthetic, 4 synthetic blends, and 4 synthetic/cotton blends) used in cloth masks intended for protection from the sars-cov-2 virus (diameter 100 ± 10 nm).The fe and Δp increased monotonically with the number of cloth layers for a lightweight flannel, suggesting that multilayered cloth masks may offer increased protection from nanometer-sized aerosol with a maximum fe dictated by breathability (i.e., Δp).filtration efficiencies of nanoscale aerosol by cloth mask materials used to slow the spread of sars-cov-2."
2020,"Konda, and others.",Aerosol Filtration Efficiency of Common Fabrics Used in Respiratory Cloth Masks,"Overall, we find that combinations of various commonly available fabrics used in cloth masks can potentially provide significant protection against the transmission of aerosol particles.aerosol filtration efficiency of common fabrics used in respiratory cloth masks."


pub_date,authors,title,fragment
2020-02-03,no and others.,Early Epidemiological and Clinical Characteristics of 28 Cases of Coronavirus Disease in South Korea,"Of the 28 cases, 16 were index cases imported from abroad, with 10 cases of secondary infection originating in korea, and the route of transmission still under investigation for 2 patients."
2020-02-03,no and others.,Contact Transmission of COVID-19 in South Korea: Novel Investigation Techniques for Tracing Contacts,"In the epidemiological investigation of an infectious disease, investigating, classifying, tracking, and managing contacts by identifying the patient’s route are important for preventing further transmission of the disease."
2020-04-06,"Nouri‐Vaskeh, and others.",Fecal transmission in COVID‐19: A potential shedding route,All rights reserved.fecal transmission in covid‐19: a potential shedding route.
2020-07-14,"Aboubakr, and others.",Stability of SARS‐CoV‐2 and other coronaviruses in the environment and on common touch surfaces and the influence of climatic conditions: A review,"The preventive measures depend mainly on the understanding of the transmission routes of this virus, its environmental stability, and its persistence on common touch surfaces."
2020-05-25,"Kotian, and others.",Knowledge and understanding among medical imaging professionals in India during the rapid rise of the covid-19 pandemic,The knowledge and understanding questionnaire consisted questions regarding the clinical characteristics and transmission routes of covid-19.
